In [84]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
%%time

# Third-party imports
import os.path as op
import pandas as pd
import great_expectations as ge

# Project imports
from ta_lib.core.api import display_as_tabs, initialize_environment

# Initialization
initialize_environment(debug=False, hide_warnings=True)

CPU times: user 80 µs, sys: 18 µs, total: 98 µs
Wall time: 102 µs


# Data

## Background

Customer is a distributor of electronic devices. They partner with manufacturers, carriers and refurbishers and sell across to  retailers. The selling price is the outcome of negotiation between sales representatives and retailers. Customer wants to understand the selling price variation and determine  optimal pricing with Machine Learning.

In [86]:
from ta_lib.core.api import create_context, list_datasets, load_dataset

In [87]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)

In [88]:
list_datasets(context)

['/raw/housing',
 '/cleaned/housing_cleaned',
 '/processed/train_x',
 '/processed/train_y',
 '/processed/test_x',
 '/processed/test_y']

In [89]:
# load datasets
house_df = load_dataset(context, '/raw/housing')

# Exploratory Analysis

Given the raw data from data ingestion, we would now like to explore and learn more details about the data.


The output of the step would be a summary report and discussion of any pertinent findings.


In [90]:
# Import the eda API
import ta_lib.eda.api as eda

## Variable summary

In [91]:
display_as_tabs([('housing', house_df.shape)])

Tabs
    [0] Str(tuple, name='housing')

In [92]:
sum1 = eda.get_variable_summary(house_df)

display_as_tabs([('house', sum1)])

Tabs
    [0] DataFrame(height=300, name='house', sizing_mode='fixed', value=                         V..., width=800)

In [93]:
house_df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.


## Merging

We can merge orders table with prod table on SKU. Let us check first-cut cardinality issues. 

### Expected data validation rules

In [94]:
verification_dict = {}
house_df = ge.from_pandas(house_df)

#### Rule 1 verification

In [95]:
verification_dict["rule_1_check"] = house_df.expect_column_values_to_be_of_type("median_income", "float64", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_1_check"]["success"]:
    print("Rule 1 passed")
else:
    print("Rule 1 failed")

Rule 1 passed


#### Rule 2 verification

In [96]:
import numpy as np

In [97]:
house_df["income_cat"] = pd.cut(house_df["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
verification_dict["rule_2_check"] = house_df.expect_column_pair_values_to_be_equal("income_cat", "median_income",mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_2_check"]["success"]:
    print("Rule 2 passed")
else:
    print("Rule 2 failed")

Rule 2 failed


### Table cardinality

In [98]:
from  ta_lib.core.api import setanalyse

setanalyse(house_df.income_cat.tolist(), house_df.median_house_value.tolist())

{'A-B': 5, 'B-A': 3842, 'AuB': 3847, 'A^B': 0}

This represents a venn diagram on two lists. Left list is `A` & right one is `B`. `A-B` implies that are five SKUs in house_df missing in product master. We can find them using unsimplied version.

Let us look at the set `A-B`.

In [99]:
missings_master_skus = setanalyse(house_df.income_cat.tolist(), house_df.median_house_value.tolist(),simplify=False)['A-B']
missings_master_skus

{1, 2, 3, 4, 5}

In [100]:
import numpy as np
print(('Records affected due to missing keys are {0} accounting to {1}% of orders').format(
    house_df.income_cat.isin(missings_master_skus).sum(),np.round(house_df.income_cat.isin(missings_master_skus).mean()*100,2)))

Records affected due to missing keys are 20640 accounting to 100.0% of orders


**Since missing keys are very less we can proceed with inner join**

**Dev NOTES**

<details>
1. Cardinality with mulitple keys: If you have more than one key use setanalyse_df. You can pass dataframes instead of lists and the key columns
2. Excess master data (if `B-A` >0 in above example) will not be concern
3. If the `A-B` is larger, please check with client for alternative data sources. In case of left join keep a stragey for imputing.


### Master uniqueness

Product master is expected to have non duplicate primary keys. Let us verify them.

## Health Analysis

Get an overview of the overall health of the dataset. This is usually quick to compute and hopefully highlights some problems to focus on.



### Summary Plot

Provides a high level summary of the dataset health.

**Watch out for:**

* too few numeric values
* high % of missing values
* high % of duplicate values
* high % of duplicate columns 

In [101]:
house_df.info()

<class 'great_expectations.dataset.pandas_dataset.PandasDataset'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   longitude           20640 non-null  float64 
 1   latitude            20640 non-null  float64 
 2   housing_median_age  20640 non-null  float64 
 3   total_rooms         20640 non-null  float64 
 4   total_bedrooms      20433 non-null  float64 
 5   population          20640 non-null  float64 
 6   households          20640 non-null  float64 
 7   median_income       20640 non-null  float64 
 8   median_house_value  20640 non-null  float64 
 9   ocean_proximity     20640 non-null  object  
 10  income_cat          20640 non-null  category
dtypes: category(1), float64(9), object(1)
memory usage: 1.6+ MB


In [102]:
sum1, plot1 = eda.get_data_health_summary(house_df, return_plot=True)

display_as_tabs([('house', plot1)])

Error: tigerml currently supports only pandas, dask and vaex

UnboundLocalError: local variable 'summary' referenced before assignment

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.

2. The missing value plot seems to indicate missing values are not present but we do have them. 

3. We are looking for duplicate observations (rows in the data). The plot shows the % of rows that are an exact replica of another row (using `df.duplicated`)

4. We are looking for duplicate features (columns in the data).

</details>

### Missing Values summary

This provides an overall view focussing on amount of missing values in the dataset.

**Watch out for:**
* A few columns have significant number of missing values 
* Most columns have significant number of missing values


In [ ]:
sum1, plot1 = eda.get_missing_values_summary(house_df, return_plot=True)

display_as_tabs([('house', plot1)])

Error: tigerml currently supports only pandas, dask and vaex

UnboundLocalError: local variable 'summary' referenced before assignment

**Dev notes:**

<details>
    
    * By default, the following are considered missing/NA values : `[np.Nan, pd.NaT, 'NA', None]`
    * additional values can be passed to tigerml (add_additional_na_values)
    * these are applied to all columns.
    
    * some of the above information can be learnt from the data discovery step (see discussion below)
    
</details>

In [ ]:
sum1 = eda.get_duplicate_columns(house_df)

display_as_tabs('house', sum1)

Error: tigerml currently supports only pandas, dask and vaex

UnboundLocalError: local variable 'out' referenced before assignment

In [ ]:
sum1 = eda.get_outliers(house_df)

display_as_tabs('house', sum1)

Error: tigerml currently supports only pandas, dask and vaex

UnboundLocalError: local variable 'out_df' referenced before assignment

## Health Analysis report

Generate a report that has all the above data in a single html. This could be useful to submit to a client

In [ ]:
from ta_lib.reports.api import summary_report

summary_report(house_df, './house.html')

Error: tigerml currently supports only pandas, dask and vaex

prod: https://drive.google.com/file/d/1TM-T5HzAYpT8_1ugM5L6Bnfxp8r3uMem/view?usp=sharing

orders: https://drive.google.com/file/d/1uvehi90v1HFtScZrtWg3pW2zpi-DFQ1Y/view?usp=sharing
    